In [2]:
# coding=utf-8
import math
train="data/train.txt"
test="data/test.txt"

In [3]:
class ItermCF:
    #载入训练数据和测试数据
    def __init__(self, basefile, testfile):
        self.datafile = basefile
        self.testfile = testfile
        self.readData()
        self.readTestData()
        
    #读取训练数据 (用户编号,物品编号,记录号)
    def readData(self):
        self.traindata = {}
        for line in open(self.datafile):
            userid, itemid, record, _ = line.split()
            self.traindata.setdefault(userid, {})
            self.traindata[userid][itemid] = int(record)
        #print(self.traindata)   
            
    #读取测试数据 (用户编号,物品编号,记录号)
    def readTestData(self):
        self.testdata = {}
        for line in open(self.testfile):
            userid, itemid, record, _ = line.split()
            self.testdata.setdefault(userid, {})
            self.testdata[userid][itemid] = int(record)
        #print(self.testdata)
            
    
    #计算物品间的相似度
    def ItemSimilarity(self):
        train = self.traindata
        C = dict()
        N = dict()
        for u, items in train.items():
            for i in items.keys():
                N.setdefault(i, 0)
                N[i] += 1
                for j in items.keys():
                    if i == j:
                        continue
                    C.setdefault(i, {})
                    C[i].setdefault(j, 0)
                    C[i][j] += 1
        self.itermSimBest = dict()  # 物品与物品之间的相似度
        for i, related_items in C.items():
            self.itermSimBest.setdefault(i, {})
            for j, cij in related_items.items():
                self.itermSimBest[i].setdefault(j, 0);
                self.itermSimBest[i][j] = cij / math.sqrt(N[i] * N[j])

    def Recommendation(self, user_id, K=8, nitem=40):
        train = self.traindata
        rank = dict()
        ru = train.get(user_id)  # 用户历史记录
        for i, pi in ru.items():
            for j, wj in sorted(self.itermSimBest[i].items(), key=lambda x: x[1], reverse=True)[0:K]:
                if j in ru:
                    continue
                rank.setdefault(j, 0)
                rank[j] += pi * wj
        return dict(sorted(rank.items(), key=lambda x: x[1], reverse=True)[0:nitem])

    def recallAndPrecision(self, test=None, k=8, nitem=10):
        train = self.traindata
        test = self.testdata
        hit = 0
        recall = 0
        precision = 0
        for user in train.keys():
            tu = test.get(user, {})
            rank = self.Recommendation(user, k, nitem)
            for item, _ in rank.items():
                if item in tu:
                    hit += 1
            recall += len(tu)
            precision += nitem
        return (hit / (recall * 1.0), hit / (precision * 1.0))


def testUserCF():
    cf = ItermCF(train,test)
    cf.ItemSimilarity()
    print ("%5s%5s%20s%20s" % ('K', 'N', "recall", 'precision'))
    for k in [5, 10, 20, 40, 80, 160]:
        for nitem in [5, 10, 15, 20]:
            recall, precision = cf.recallAndPrecision(k=k, nitem=nitem)
            print  ("%5d%5d%19.3f%%%19.3f%%" % (k, nitem, recall * 100, precision * 100))


if __name__ == '__main__':
    testUserCF()

    K    N              recall           precision
    5    5              5.575%             23.648%
    5   10              9.740%             20.657%
    5   15             13.335%             18.855%
    5   20             16.370%             17.359%
   10    5              5.585%             23.690%
   10   10              9.955%             21.113%
   10   15             13.780%             19.484%
   10   20             17.230%             18.271%
   20    5              5.540%             23.499%
   20   10              9.905%             21.007%
   20   15             13.690%             19.357%
   20   20             17.065%             18.097%
   40    5              5.395%             22.884%
   40   10              9.700%             20.573%
   40   15             13.300%             18.805%
   40   20             16.445%             17.439%
   80    5              5.430%             23.033%
   80   10              9.670%             20.509%
   80   15             12.975% 